In [71]:
import requests
from sklearn.model_selection import train_test_split
import pandas as pd
from random import randint
import time
from arize.api import Client
from arize.utils.types import ModelTypes, Environments
from getpass import getpass

In [73]:
# replace host address with an actual one
URL = "http://test2-yatai.13.39.60.211.sslip.io/classify"

In [74]:
SPACE_KEY = "fa2cbad"
if 'API_KEY' not in locals():
    API_KEY = getpass("Enter your Arize Api Key: ")

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

In [75]:
# building train dataset for prediction simulation

training_df = pd.read_parquet(f"../data/loan_table_training.parquet")  # load train dataset
target = "loan_status"
x = training_df[
    training_df.columns.drop(target)
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("zipcode")
    .drop("dob_ssn")
]
X = x.reindex(sorted(x.columns), axis=1)
y = training_df.loc[:, target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=1234
)

In [ ]:
while True:

    n = randint(0, X_test.shape[0] - 1)
    line = X_test.iloc[[n]]

    request = requests.post(
        url=URL,
        headers={"content-type": "application/json"},
        data=line.to_json(orient='records')
    )
    j = request.json()
    result, prediction_id, model_version = str(j['prediction']), str(j['prediction_id']), str(j['model_version'])

    # In reality we must wait for 6-9 months in case of credit scoring model
    actual_result = str(y_test[line.index].iloc[0])

    future = arize_client.log(
        prediction_id=prediction_id,
        actual_label=actual_result,
        model_id="credit_score:credit_score_prediction",
        model_version=model_version,
        model_type=ModelTypes.SCORE_CATEGORICAL,
        environment=Environments.PRODUCTION,
        prediction_label=result,
        features=line.to_dict('records')[0] # hack: in real case we must not log features second time
    )

    if future.result().status_code != 200:
        print(f'Logging failed with status code {result.status_code} and message "{result.text}"')

    time.sleep(1)